In [1]:
from allensdk.brain_observatory.behavior.behavior_ophys_session import BehaviorOphysSession

ophys_experiment_id = 856938751
sdk_session = BehaviorOphysSession.from_lims(ophys_experiment_id)
sdk_trial = sdk_session.trials.iloc[34]

sdk_trial[['change_time', 'change_frame', 'reward_time', 'lick_times', 'response_latency']]

/allen/programs/braintv/workgroups/nc-ophys/Doug/code/AllenSDK/allensdk/brain_observatory/session_api_utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/allen/programs/braintv/workgroups/nc-ophys/Doug/code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam2_exposure', 'cam1_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()


change_time                                                463.038538
change_frame                                                  27299.0
reward_time                                                463.672388
lick_times          [463.6723878878255, 463.7725078878255, 464.022...
response_latency                                              0.63385
Name: 34, dtype: object

In [4]:
import allensdk

In [5]:
allensdk.__version__

'2.6.0'

In [16]:
from visual_behavior.translator.foraging2 import data_to_change_detection_core
import pandas as pd
from visual_behavior.translator.core import create_extended_dataframe
import visual_behavior.database as db
import visual_behavior.utilities as vbu

def get_behavior_session_id_from_ophys_experiment_id(ophys_experiment_id):
    '''sql query to get behavior session ID given an ophys experiment ID'''
    query = '''
        select bs.id
        from ophys_experiments as oe
        join ophys_sessions as os on oe.ophys_session_id = os.id
        join behavior_sessions as bs on os.foraging_id = bs.foraging_id
        where oe.id = {}
    '''
    return db.lims_query(query.format(ophys_experiment_id))

def get_ophys_session_id_from_ophys_experiment_id(ophys_experiment_id):
    '''sql query to get ophys session ID given an ophys experiment ID'''
    query = '''
        select ophys_session_id
        from ophys_experiments
        where id = {}
    '''
    return db.lims_query(query.format(ophys_experiment_id))

def open_vba_session(behavior_session_id, time=None):
    '''convenience function to open vba session'''
    pkl_path = db.get_pkl_path(behavior_session_id)
    data = pd.read_pickle(pkl_path)
    core_data = data_to_change_detection_core(data, time=time)
    return core_data

def get_sync_stim_timestamps(ophys_session_id=None, ophys_experiment_id=None):
    '''get stimulus timestamps from sync given an ophys session ID'''
    if ophys_experiment_id:
        ophys_session_id = get_ophys_session_id_from_ophys_experiment_id(ophys_experiment_id)
    if ophys_session_id:
        wkf = db.get_well_known_files(ophys_session_id, 'OphysSession')
    sync_filepath = ''.join([wkf.loc['OphysRigSync']['storage_directory'], wkf.loc['OphysRigSync']['filename']])
    sync_data = vbu.get_sync_data(sync_filepath)
    return sync_data['stim_vsync_falling']

## open core data, passing sync time to replace PKL timestamps
core_data = open_vba_session(
    get_behavior_session_id_from_ophys_experiment_id(ophys_experiment_id),
    time=get_sync_stim_timestamps(ophys_experiment_id = ophys_experiment_id)
)
## build out the extended trial dataframe, which does some time rebasing and other trial labeling
extended_trials = create_extended_dataframe(**core_data)

In [25]:
extended_trials.iloc[34][['change_time', 'change_frame', 'lick_times', 'lick_frames', 'reward_times', 'reward_frames', 'response_latency']]

change_time                                                   463.017
change_frame                                                    27299
lick_times          [463.65082, 463.75094, 464.00109, 464.11785, 4...
lick_frames         [27337, 27343, 27358, 27365, 27371, 27379, 273...
reward_times                                              [463.65082]
reward_frames                                                 [27337]
response_latency                                              0.63385
Name: 34, dtype: object

In [22]:
extended_trials.columns

Index(['index', 'lick_times', 'auto_rewarded', 'cumulative_volume',
       'cumulative_reward_number', 'reward_volume', 'reward_times',
       'reward_frames', 'rewarded', 'optogenetics', 'response_type',
       'response_time', 'response_latency', 'change_frame', 'change_time',
       'starttime', 'startframe', 'trial_length', 'scheduled_change_time',
       'endtime', 'endframe', 'initial_image_category', 'initial_image_name',
       'change_image_name', 'change_image_category', 'change_ori',
       'change_contrast', 'initial_ori', 'initial_contrast', 'delta_ori',
       'mouse_id', 'response_window', 'task', 'stage', 'session_duration',
       'user_id', 'LDT_mode', 'blank_screen_timeout', 'stim_duration',
       'blank_duration_range', 'prechange_minimum', 'stimulus_distribution',
       'stimulus', 'distribution_mean', 'computer_name',
       'behavior_session_uuid', 'startdatetime', 'date', 'year', 'month',
       'day', 'hour', 'dayofweek', 'number_of_rewards', 'rig_id', 'trial

In [12]:
get_sync_stim_timestamps(ophys_experiment_id = ophys_experiment_id)

array([   7.62935,    7.64597,    7.66274, ..., 4515.04042, 4515.05707,
       4515.07375])

In [26]:
463.055 - 463.017

0.038000000000010914

In [27]:
core_data['rewards']

,frame,time,volume,lickspout
7,19254,328.82413,0.005,None
16,20918,356.58012,0.005,None
21,22808,388.10584,0.005,None
23,23483,399.36503,0.005,None
26,24653,418.88095,0.005,None
...,...,...,...,...
593,199659,3349.43924,0.007,None
601,202095,3390.07239,0.007,None
613,206237,3459.16212,0.007,None
634,213258,3576.27433,0.007,None


In [28]:
sdk_session.rewards

,volume,autorewarded
timestamps,,
327.923551,0.005,True
355.687041,0.005,True
387.206521,0.005,True
398.466333,0.005,True
417.986026,0.005,True
...,...,...
3348.549266,0.007,False
3389.178350,0.007,False
3458.267680,0.007,False


In [42]:
merged_rewards = pd.merge(
    left = core_data['rewards'].reset_index(drop=True)[['frame','time']].rename(columns={'frame':'vba_frame', 'time':'vba_time'}),
    right = sdk_session.rewards.reset_index()[['timestamps']].rename(columns={'timestamps':'sdk_time'}),
    left_index = True,
    right_index = True
)
merged_rewards['time_diff'] = merged_rewards['vba_time'] - merged_rewards['sdk_time']
merged_rewards

,vba_frame,vba_time,sdk_time,time_diff
0,19254,328.82413,327.923551,0.900579
1,20918,356.58012,355.687041,0.893079
2,22808,388.10584,387.206521,0.899319
3,23483,399.36503,398.466333,0.898697
4,24653,418.88095,417.986026,0.894924
...,...,...,...,...
77,199659,3349.43924,3348.549266,0.889974
78,202095,3390.07239,3389.178350,0.894040
79,206237,3459.16212,3458.267680,0.894440
80,213258,3576.27433,3575.375358,0.898972


In [44]:
sdk_session.licks

,time
0,14.401231
1,14.901623
2,20.039135
3,20.256319
4,24.409378
...,...
2984,4098.684009
2985,4099.100986
2986,4099.534686
2987,4107.507861


In [46]:
core_data['licks']

,frame,time
0,406,14.40120
1,436,14.90159
2,744,20.03910
3,757,20.25629
4,1006,24.40935
...,...,...
2984,244577,4098.68398
2985,244602,4099.10096
2986,244628,4099.53466
2987,245106,4107.50783


In [47]:
merged_licks = pd.merge(
    left = core_data['licks'].reset_index(drop=True)[['frame','time']].rename(columns={'frame':'vba_frame', 'time':'vba_time'}),
    right = sdk_session.licks.rename(columns={'time':'sdk_time'}),
    left_index = True,
    right_index = True
)
merged_licks['time_diff'] = merged_licks['vba_time'] - merged_licks['sdk_time']
merged_licks

,vba_frame,vba_time,sdk_time,time_diff
0,406,14.40120,14.401231,-0.000031
1,436,14.90159,14.901623,-0.000033
2,744,20.03910,20.039135,-0.000035
3,757,20.25629,20.256319,-0.000029
4,1006,24.40935,24.409378,-0.000028
...,...,...,...,...
2984,244577,4098.68398,4098.684009,-0.000029
2985,244602,4099.10096,4099.100986,-0.000026
2986,244628,4099.53466,4099.534686,-0.000026
2987,245106,4107.50783,4107.507861,-0.000031


In [52]:
core_data['visual_stimuli'][core_data['visual_stimuli']['image_name'].str.startswith('im')]

,orientation,image_name,image_category,frame,end_frame,time,duration,stop_frame
0,NaN,im065,im065,17987,18002.0,307.65685,0.25021,NaN
1,NaN,im065,im065,18032,18047.0,308.40747,0.25022,NaN
2,NaN,im065,im065,18077,18092.0,309.15810,0.25058,NaN
3,NaN,im065,im065,18122,18137.0,309.90897,0.25023,NaN
4,NaN,im065,im065,18167,18181.0,310.67601,0.23352,NaN
...,...,...,...,...,...,...,...,...
4630,NaN,im069,im069,233330,233345.0,3911.08089,0.25020,NaN
4631,NaN,im069,im069,233375,233390.0,3911.83151,0.25018,NaN
4632,NaN,im069,im069,233420,233435.0,3912.58210,0.25020,NaN
4633,NaN,im069,im069,233465,233480.0,3913.33271,0.25049,NaN


In [53]:
sdk_session.stimulus_presentations.query('omitted == False')

,duration,end_frame,image_index,image_name,image_set,index,omitted,orientation,phase,spatial_frequency,start_frame,start_time,stop_time
stimulus_presentations_id,,,,,,,,,,,,,
0,0.25021,18002.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,0,False,NaN,NaN,NaN,17987,307.678418,307.928628
1,0.25022,18047.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,1,False,NaN,NaN,NaN,18032,308.429038,308.679258
2,0.25058,18092.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,2,False,NaN,NaN,NaN,18077,309.179668,309.430248
3,0.25023,18137.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,3,False,NaN,NaN,NaN,18122,309.930538,310.180768
4,0.23352,18181.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,4,False,NaN,NaN,NaN,18167,310.697578,310.931098
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,0.25020,233345.0,7,im069,Natural_Images_Lum_Matched_set_training_2017.0...,4630,False,NaN,NaN,NaN,233330,3911.102458,3911.352658
4795,0.25018,233390.0,7,im069,Natural_Images_Lum_Matched_set_training_2017.0...,4631,False,NaN,NaN,NaN,233375,3911.853078,3912.103258
4796,0.25020,233435.0,7,im069,Natural_Images_Lum_Matched_set_training_2017.0...,4632,False,NaN,NaN,NaN,233420,3912.603668,3912.853868


In [64]:
merged_stimuli = pd.merge(
    (core_data['visual_stimuli'][core_data['visual_stimuli']['image_name'].str.startswith('im')][['frame','time']]
     .rename(columns = {'frame':'vba_frame','time':'vba_time'})), # get only visual stimuli, not movie stimuli
    (sdk_session.stimulus_presentations.query('omitted == False')[['start_frame', 'start_time']]
     .rename(columns = {'start_frame':'sdk_frame','start_time':'sdk_time'})
     .reset_index(drop=True)
    ), # get only non-omitted visual stimuli
    left_index = True,
    right_index = True
)
merged_stimuli['time_diff'] = merged_stimuli['sdk_time'] - merged_stimuli['vba_time']
merged_stimuli

,vba_frame,vba_time,sdk_frame,sdk_time,time_diff
0,17987,307.65685,17987,307.678418,0.021568
1,18032,308.40747,18032,308.429038,0.021568
2,18077,309.15810,18077,309.179668,0.021568
3,18122,309.90897,18122,309.930538,0.021568
4,18167,310.67601,18167,310.697578,0.021568
...,...,...,...,...,...
4630,233330,3911.08089,233330,3911.102458,0.021568
4631,233375,3911.83151,233375,3911.853078,0.021568
4632,233420,3912.58210,233420,3912.603668,0.021568
4633,233465,3913.33271,233465,3913.354278,0.021568


In [62]:
(sdk_session.stimulus_presentations.query('omitted == False')[['start_frame', 'start_time']]
     .rename(columns = {'start_frame':'sdk_frame','start_time':'sdk_time'}))

,sdk_frame,sdk_time
stimulus_presentations_id,,
0,17987,307.678418
1,18032,308.429038
2,18077,309.179668
3,18122,309.930538
4,18167,310.697578
...,...,...
4794,233330,3911.102458
4795,233375,3911.853078
4796,233420,3912.603668


In [65]:
get_ophys_session_id_from_ophys_experiment_id(856938751)

856295914

In [67]:
dir(sdk_session.api)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_behavior_stimulus_file',
 '_get_ids',
 '_process_ophys_plane_timestamps',
 'behavior_session_id',
 'cache_clear',
 'from_foraging_id',
 'get_age',
 'get_api_list_by_container_id',
 'get_average_intensity_projection_image_file',
 'get_average_projection',
 'get_behavior_session_id',
 'get_behavior_session_uuid',
 'get_behavior_stimulus_file',
 'get_birth_date',
 'get_cell_roi_ids',
 'get_cell_specimen_table',
 'get_containers_df',
 'get_corrected_fluorescence_traces',
 'get_demix_file',
 'get_dff_file',
 'get_dff_traces',
 'get_driver_line',
 'get_experiment_container_i

In [68]:
sdk_session.api.get_extended_trials()

RuntimeError: Should not have been possible